In [863]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import math
import os

maxAcc = 0.0
maxIter = 0
C_Lambda = 0.03
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10
M = 10
PHI = []
IsSynthetic = False

In [864]:
def create_pair_data_concat(df, df_pair):
    df.rename(columns={'img_id': 'img_id_A'}, inplace=True)
    df_new = pd.merge(df_pair, df, on="img_id_A")
    df.rename(columns={'img_id_A': 'img_id_B'}, inplace=True)
    df_new = pd.merge(df_new, df, on="img_id_B")
    df.rename(columns={'img_id_B': 'img_id'}, inplace=True)
    return df_new  

In [865]:
def create_pair_data_subtract(df):
    for i in range(1,10):
        df[["f"+str(i)]] = df[["f"+str(i)+"_x"]] - df[["f"+str(i)+"_y"]].values
    for i in range(1,10):
        df = df.drop(columns=["f"+str(i)+"_x" , "f"+str(i)+"_y"], axis=1)
    return df

In [866]:
def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    return t

def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

In [867]:
def generate_raw_data(df, df_diff, df_same, feature_operation):
    df_same_concat = create_pair_data_concat(df, df_same)
    df_diff_concat = create_pair_data_concat(df, df_diff)
    df_diff_concat = df_diff_concat.sample(n=df_same_concat.shape[0])
    df_final = pd.concat([df_same_concat, df_diff_concat])
    df_final = df_final.iloc[np.random.permutation(len(df_final))]
    if feature_operation == "Subtract":
        df_final = create_pair_data_subtract(df_final)
    df_target = df_final['target']
    df_final.drop(['img_id_A','img_id_B','target' ], axis=1, inplace=True)
    return df_final, df_target

In [868]:
def sigmoid_func(W, X):
    return 1.0/(1 + np.exp(-np.dot(X, np.transpose(W))))

In [869]:
# ------------- Implementation for Stochastic Gradient Descent --------------- #
def generate_ERMS(TrainingTarget, TrainingData, ValData, TestData, ValDataAct, TestDataAct, dataType):
    if dataType == "GSC":
        range_loop = 400
    else:
        range_loop = TrainingTarget.shape[0]
    W_Now = np.ones(TrainingData.shape[0])
    La           = 2
    learningRate = 0.01
    L_Erms_Val   = []
    L_Erms_TR    = []
    L_Erms_Test  = []
    W_Mat        = []
    
    for i in range(0,400):
        step1 = sigmoid_func(W_Now, np.transpose(TrainingData))
        step2 = np.subtract(step1, TrainingTarget)
        Delta_E_D =  np.dot(TrainingData, step2)/TrainingTarget.shape[0]
        La_Delta_E_W  = np.dot(La,W_Now)
        Delta_E       = np.add(Delta_E_D,La_Delta_E_W) 
        Delta_W       = -np.dot(learningRate,Delta_E)
        W_T_Next      = W_Now + Delta_W
        W_Now         = W_T_Next

        #-----------------TrainingData Accuracy---------------------#
        TR_TEST_OUT   = sigmoid_func(W_T_Next,np.transpose(TrainingData)) 
        Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
        L_Erms_TR.append(float(Erms_TR.split(',')[0]))

        #-----------------ValidationData Accuracy---------------------#
        VAL_TEST_OUT  = sigmoid_func(W_T_Next,np.transpose(ValData)) 
        Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
        L_Erms_Val.append(float(Erms_Val.split(',')[0]))

        #-----------------TestingData Accuracy---------------------#
        TEST_OUT      = sigmoid_func(W_T_Next,np.transpose(TestData)) 
        Erms_Test = GetErms(TEST_OUT,TestDataAct)
        L_Erms_Test.append(float(Erms_Test.split(',')[0]))
        
    print ('----------Gradient Descent Solution--------------------')
    print ("Accuracy Training   = " + str(np.around(max(L_Erms_TR),5)))
    print ("Accuracy Validation = " + str(np.around(max(L_Erms_Val),5)))
    print ("Accuracy Testing    = " + str(np.around(max(L_Erms_Test),5)))  
    

In [870]:
def train_model(RawData, RawTarget, dataType):
    TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
    TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
    
    ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
    ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
    
    TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
    TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
    
    generate_ERMS(TrainingTarget, TrainingData, ValData, TestData, ValDataAct, TestDataAct, dataType)

In [871]:
df_hof = pd.read_csv("HOF/HumanObserved-Features-Data.csv",index_col=0)
df_diff_hof = pd.read_csv("HOF/diffn_pairs.csv",index_col=0)
df_same_hof = pd.read_csv("HOF/same_pairs.csv",index_col=0)

print("-----------------------HOF : Concat--------------------")
df_final, df_target = generate_raw_data(df_hof, df_diff_hof, df_same_hof, "Concat")
RawData = np.transpose(df_final.as_matrix())
RawTarget = df_target.as_matrix()
train_model(RawData, RawTarget, "HOF")

print("-----------------------HOF : Subtract--------------------")
df_final, df_target = generate_raw_data(df_hof, df_diff_hof, df_same_hof, "Subtract")
RawData = np.transpose(df_final.as_matrix())
RawTarget = df_target.as_matrix()
train_model(RawData, RawTarget,"HOF")

df_gsc = pd.read_csv("GSC/GSC-Features.csv")
df_diff_gsc = pd.read_csv("GSC/diffn_pairs.csv")
df_same_gsc = pd.read_csv("GSC/same_pairs.csv")

print("-----------------------GSC : Concat--------------------")
df_final, df_target = generate_raw_data(df_gsc, df_diff_gsc, df_same_gsc, "Concat")
uniques = df_final.apply(lambda x: x.nunique())
df_final = df_final.drop(uniques[uniques==1].index, axis=1)
RawData = np.transpose(df_final.as_matrix())
RawTarget = df_target.as_matrix()
train_model(RawData, RawTarget, "GSC")

print("-----------------------GSC : Subtract--------------------")
df_final, df_target = generate_raw_data(df_gsc, df_diff_gsc, df_same_gsc, "Subtract")
uniques = df_final.apply(lambda x: x.nunique())
df_final = df_final.drop(uniques[uniques==1].index, axis=1)
RawData = np.transpose(df_final.as_matrix())
RawTarget = df_target.as_matrix()
train_model(RawData, RawTarget, "GSC")

-----------------------HOF : Concat--------------------


/Users/atrayeenag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
/Users/atrayeenag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


----------Gradient Descent Solution--------------------
Accuracy Training   = 57.109
Accuracy Validation = 54.43038
Accuracy Testing    = 56.05096
-----------------------HOF : Subtract--------------------


/Users/atrayeenag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/Users/atrayeenag/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


----------Gradient Descent Solution--------------------
Accuracy Training   = 55.60821
Accuracy Validation = 50.0
Accuracy Testing    = 54.77707
